In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from HFCs_bmimPF6_PR import configuration

# Load data from csv
data = pd.read_csv('HFCSIL_g.csv')

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(298.15)
    m.fs.state_block.pressure.fix(100000)
    m.fs.state_block.mole_frac_comp["R32"].fix(0.02)
    m.fs.state_block.mole_frac_comp["R125"].fix(0.02)
    m.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.96)
    
    m.fs.properties.PR_kappa_A['R32', 'R125'].fix(-0.01)
    m.fs.properties.PR_kappa_A['R125', 'R32'].fix(-0.01)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.02450)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.07543)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(0.28636)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(-0.0035)
    
#     m.fs.properties.PR_kappa_A['R32', 'R125'].fix(2.201)
#     m.fs.properties.PR_kappa_A['R125', 'R32'].fix(-0.499549)
#     m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.0451)
#     m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.452889)
#     m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(1.98424)
#     m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(0.061815)
    
#     m.fs.properties.bmimPF6.temperature_crit.fix(900)
#     m.fs.properties.bmimPF6.pressure_crit.fix(25e5)
#     m.fs.properties.bmimPF6.omega.fix(0.5)
    
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
    m.fs.state_block.temperature.fix(float(data["temperature"]))
#     m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp["R125"].unfix()
    m.fs.state_block.mole_frac_comp["R32"].unfix()
    m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))   
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"])) 
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data["x_IL"])) 
    
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['R32', 'R125'].setlb(-10)
    m.fs.properties.PR_kappa_A['R32', 'R125'].setub(10)

    m.fs.properties.PR_kappa_A['R125', 'R32'].setlb(-10)
    m.fs.properties.PR_kappa_A['R125', 'R32'].setub(10)

    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setlb(-10)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setub(10)

    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setlb(-10)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setub(10)
    
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setlb(-10)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setub(10)

    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setlb(-10)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setub(10)
    
#     m.fs.properties.bmimPF6.temperature_crit.setlb(800)
#     m.fs.properties.bmimPF6.temperature_crit.setub(1500)
    
#     m.fs.properties.bmimPF6.pressure_crit.setlb(1e5)
#     m.fs.properties.bmimPF6.pressure_crit.setub(5e7)
    
    m.fs.properties.bmimPF6.omega.setlb(0.0001)
    m.fs.properties.bmimPF6.omega.setub(0.9999)
    
    # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 100000, "x_R32":0.7312, "x_IL":0.7312,  
             "x_R125":0.2688, "y_R125":0.2501, "y_R32":0.7499, "y_IL":0.7312}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

2022-02-03 15:28:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 1


In [7]:
variable_name = [
#                 "fs.properties.bmimPF6.temperature_crit",
#                  "fs.properties.bmimPF6.pressure_crit",
#                  "fs.properties.bmimPF6.omega",
                "fs.properties.PR_kappa_A['R32', 'R125']",
                "fs.properties.PR_kappa_A['R125', 'R32']",
                "fs.properties.PR_kappa_A['R32', 'bmimPF6']",
                "fs.properties.PR_kappa_A['bmimPF6', 'R32']",
                "fs.properties.PR_kappa_A['bmimPF6', 'R125']",
                "fs.properties.PR_kappa_A['R125', 'bmimPF6']"]

In [8]:
def SSE(m, data):
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
    return expr*1e-9

# def SSE(m, data):
#     expr = ((float(data["y_R125"]) -
#              m.fs.state_block.mole_frac_phase_comp["Vap", "R125"])**2)


In [9]:
solver_options = {'tol': 1e-5}
pest = parmest.Estimator(PR_model, data, variable_name, SSE, solver_options)
# solver_options = {'tol': 1e-6, 'max_iter':100}

obj_value, parameters, a = pest.theta_est(calc_cov=True)

2022-02-03 15:28:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-03 15:28:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-03 15:28:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-03 15:29:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-03 15:29:03 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-03 15:29:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-03 15:29:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-03 15:29:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-03 15:29:11 [INFO] idaes

  59  1.1402837e+01 6.17e-01 7.32e+08  -1.0 9.39e+03  -4.4 4.08e-01 3.96e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  60  1.1421056e+01 5.73e-01 6.87e+08  -1.0 2.60e+04  -4.8 6.31e-02 6.08e-02f  4
  61  1.1448195e+01 2.66e-01 3.29e+08  -1.0 2.01e+04  -4.4 1.80e-01 3.77e-01h  1
  62  1.1611553e+01 3.24e-01 3.24e+08  -1.0 2.07e+04  -4.9 2.19e-01 4.36e-01h  1
  63  1.1613743e+01 3.22e-01 3.13e+08  -1.0 8.97e+03  -5.4 4.32e-02 6.29e-03h  1
  64  1.1306110e+01 4.49e-01 1.57e+09  -1.0 7.95e+03  -4.9 7.54e-03 9.78e-01f  1
  65  1.1308558e+01 4.46e-01 1.92e+09  -1.0 3.72e+04  -5.4 2.24e-02 6.89e-03h  1
  66  1.1026351e+01 5.52e-01 1.47e+09  -1.0 3.20e+04  -5.9 1.72e-02 1.76e-01f  2
  67  1.0968828e+01 5.34e-01 1.13e+09  -1.0 1.29e+05  -6.4 1.34e-01 4.57e-02f  3
  68  1.0899253e+01 5.23e-01 1.84e+09  -1.0 3.08e+04  -6.0 2.21e-01 4.58e-02f  3
  69  1.0882464e+01 5.17e-01 1.48e+09  -1.0 1.02e+05  -6.4 1.83e-01 1.26e-02h  2
iter    objective    inf_pr 

In iteration 175, 1 Slack too small, adjusting variable bound
 176r 1.1304461e+00 4.19e+01 6.42e+15   1.6 1.95e+01   3.4 1.94e-02 4.45e-03f  3
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
In iteration 176, 1 Slack too small, adjusting variable bound
 177r 1.1304461e+00 4.28e+01 6.35e+15   1.6 1.46e+01   3.9 1.38e-01 4.56e-03h  6
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
In iteration 177, 1 Slack too small, adjusting variable bound
 178r 1.1304461e+00 

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
In iteration 190, 1 Slack too small, adjusting variable bound
 191r 1.1304461e+00 6.18e+01 4.45e+15   1.6 1.55e+01   4.4 3.92e-04 4.48e-03f  4
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
In iteration 191, 1 Slack too small, adjusting variable bound
 192r 1.1304461e+00 6.25e+01 4.41e+15   1.6 9.43e+00   4.8 8.79e-02 4.29e-03f  6
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
In iteration 192, 1 Slack too small, adjusting variable bound
 193r 1.1304461e+00 

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
In iteration 211, 1 Slack too small, adjusting variable bound
 212r 1.1304461e+00 9.53e+01 7.04e+14   1.6 5.99e+00   3.4 6.69e-01 1.05e-02f  7
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
In iteration 212, 1 Slack too small, adjusting variable bound
 213r 1.1304461e+00 9.65e+01 4.19e+15   1.6 5.39e+00   3.8 1.00e+00 8.30e-03f  6
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Err

...

 843r 1.1304461e+00 1.21e+02 6.01e+03  -4.4 8.47e-01   3.1 9.91e-01 1.00e+00f  1
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
In iteration 843, 1 Slack too small, adjusting variable bound
 844r 1.1304461e+00 1.19e+02 5.32e+03  -4.4 7.62e+00   2.1 1.32e-01 9.80e-01f  1
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
In iteration 844, 1 Slack too small, adjusting variable bound
 845r 1.1304461e+00 1.19e+02 6.08e+03  -4.4 6.86e+01   1.2 3.94e-01 9.19e-06f  1
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 

In iteration 884, 1 Slack too small, adjusting variable bound
 885r 1.1304451e+00 9.22e+01 1.06e+11  -4.4 5.29e-01   3.3 3.87e-03 8.23e-05f  1
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
In iteration 885, 1 Slack too small, adjusting variable bound
 886r 1.1304451e+00 9.22e+01 1.04e+11  -4.4 1.79e+00   2.8 2.79e-04 2.62e-02f  1
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
In iteration 886, 1 Slack too small, adjusting variable bound
 887r 1.1304451e+00 9.22e+01 1.03e+11  -4.4 6.01e-01   3.2 1.01e-02 3.91e-03f  1
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
In iteration 887, 1 Slack too small, adjusting variable bound
 888r 1.1304451e+00 9.22e+01 1.03e+11  -4.4 2.32e+00   2.7 1.59e-03 9.03e-07f  1
Scaling factors are invalid - setting them all to 1.
Scal

In iteration 911, 1 Slack too small, adjusting variable bound
 912r 1.1304451e+00 8.97e+01 3.69e+08  -4.4 9.84e-01   3.0 1.00e+00 1.00e+00f  1
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
In iteration 912, 1 Slack too small, adjusting variable bound
 913r 1.1304451e+00 8.95e+01 3.06e+08  -4.4 3.79e-01   3.5 1.00e+00 2.28e-01f  1
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
In iteration 913, 1 Slack too small, adjusting variable bound
 914r 1.1304451e+00 8.95e+01 3.06e+08  -4.4 1.69e+00   3.0 1.38e-01 3.58e-06f  1
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
In iteration 914, 1 Slack too small, adjusting variable bound
 915r 1.1304451e+00 8.87e+01 1.65e+08  -4.4 4.23e-01   3.4 1.00e+00 1.00e+00f  1
Scal

In iteration 954, 1 Slack too small, adjusting variable bound
 955r 1.1304451e+00 7.80e+01 1.88e+14  -6.5 5.11e-01   3.3 1.00e+00 1.37e-02f  1
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
In iteration 955, 1 Slack too small, adjusting variable bound
 956r 1.1304451e+00 7.80e+01 1.87e+14  -6.5 1.53e+00   2.8 4.94e-03 8.47e-05f  1
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
In iteration 956, 1 Slack too small, adjusting variable bound
 957r 1.1304451e+00 7.79e+01 1.55e+14  -6.5 5.75e-01   3.2 1.00e+00 2.00e-01f  1
Scaling factors are invalid - setting them all to 1.
Scaling factors are invalid - setting them all to 1.
In iteration 957, 1 Slack too small, adjusting variable bound
 958r 1.1304451e+00 7.77e+01 1.42e+14  -6.5 1.72e+00   2.8 9.86e-02 9.66e-02f  1
Scaling factors are invalid - setting them all to 1.
Scal

 998  5.1426268e-01 1.59e+01 8.52e+10  -8.6 3.61e+04    -  1.53e-01 2.87e-01h  1
 999  1.5088178e-01 1.96e+01 8.07e+10  -8.6 2.92e+04    -  6.98e-02 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1000  1.4898267e-01 1.90e+01 9.42e+10  -8.6 1.20e+04    -  5.81e-04 4.49e-02h  1
1001  1.1556918e-01 1.83e+01 1.35e+11  -8.6 5.59e+03    -  1.71e-08 1.00e+00h  1
1002  1.1537369e-01 1.82e+01 5.99e+10  -8.6 2.68e+03    -  5.09e-02 1.03e-02h  1
1003  9.7725203e-02 1.63e+01 5.68e+10  -8.6 2.76e+03    -  2.99e-08 1.00e+00h  1
1004  9.7496754e-02 1.61e+01 5.61e+10  -8.6 2.09e+03    -  2.65e-01 1.63e-02h  1
1005  8.1380899e-02 1.44e+01 5.31e+10  -8.6 2.56e+03    -  8.75e-08 1.00e+00h  1
1006  8.0291543e-02 1.37e+01 5.11e+10  -8.6 2.14e+03    -  6.21e-02 8.34e-02h  1
1007  6.8104453e-02 1.16e+01 2.14e+11  -8.6 2.05e+03    -  8.91e-09 1.00e+00h  1
1008  6.8104451e-02 7.81e+00 4.35e+10  -8.6 1.26e-01   0.2 1.54e-05 1.00e+00h  1
1009  6.8104451e-02 3.67e+00

In [10]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 0.068104

The values for the parameters are as follows:
fs.properties.PR_kappa_A[R32,R125] = -4.608945113113342
fs.properties.PR_kappa_A[R125,R32] = -0.6739716274150072
fs.properties.PR_kappa_A[R32,bmimPF6] = -0.08770802172065327
fs.properties.PR_kappa_A[bmimPF6,R32] = -2.5651660304855377
fs.properties.PR_kappa_A[bmimPF6,R125] = 1.07672647046483
fs.properties.PR_kappa_A[R125,bmimPF6] = 0.05351308456527874


In [11]:
print(large_residuals_set(pest.ef_instance))

ComponentSet([])


In [12]:
print("covariance_matrix",a)

covariance_matrix                                         fs.properties.PR_kappa_A[R32,R125]  \
fs.properties.PR_kappa_A[R32,R125]                           -1.111652e+06   
fs.properties.PR_kappa_A[R125,R32]                            8.719787e+04   
fs.properties.PR_kappa_A[R32,bmimPF6]                        -1.327359e+00   
fs.properties.PR_kappa_A[bmimPF6,R32]                        -5.343468e+01   
fs.properties.PR_kappa_A[bmimPF6,R125]                        2.303243e+05   
fs.properties.PR_kappa_A[R125,bmimPF6]                        2.687869e+03   

                                        fs.properties.PR_kappa_A[R125,R32]  \
fs.properties.PR_kappa_A[R32,R125]                            87197.870693   
fs.properties.PR_kappa_A[R125,R32]                            -1221.608331   
fs.properties.PR_kappa_A[R32,bmimPF6]                             0.336946   
fs.properties.PR_kappa_A[bmimPF6,R32]                            14.464716   
fs.properties.PR_kappa_A[bmimPF6,R125]       

In [13]:
eigvals, eigvecs = np.linalg.eig(a)

In [14]:
eigvals

array([-1.16196307e+06,  1.37805532e+05,  2.46741134e+00,  2.53369405e-01,
        1.86428646e-05,  2.65797461e-05])

In [15]:
eigvecs

array([[-9.79711350e-01,  1.62527947e-01,  5.87896289e-02,
        -1.01457899e-01, -5.63432917e-04,  3.05734637e-04],
       [ 8.08271553e-02, -2.14978105e-01,  4.87795930e-01,
        -8.42186550e-01, -4.67937617e-03,  2.53674179e-03],
       [-1.02040775e-06, -7.50153356e-06,  1.86054562e-02,
         1.64816618e-02, -5.88373330e-01,  8.08206594e-01],
       [-4.05833504e-05, -3.28061547e-04,  8.65100411e-01,
         5.01005326e-01,  1.52935767e-02, -1.89983976e-02],
       [ 1.83379551e-01,  9.62934401e-01,  9.92623886e-02,
        -1.70732904e-01,  8.48613698e-03,  7.38371873e-03],
       [ 2.13999703e-03,  1.12389058e-02,  1.00221190e-03,
         6.35844446e-04, -8.08386481e-01, -5.88540665e-01]])

In [16]:
logeigen = np.log(eigvals)

C:\Users\agarc\AppData\Local\Temp/ipykernel_2528/2073555487.py:1: RuntimeWarning: invalid value encountered in log
  logeigen = np.log(eigvals)


In [17]:
print(logeigen)

[         nan  11.83359878   0.90316956  -1.37290675 -10.89004708
 -10.53536106]
